In [0]:
%load_ext autoreload
%autoreload 2

# Importando módulo de seguimiento
import sys
import importlib
sys.path.append('/Workspace/Users/luisfdiaz@bcp.com.pe/ScoreGuardian')

import core.SegScore as rmm
importlib.reload(rmm)
 
# Librerias y fuciones generales
from pyspark.sql.functions import date_format, expr, to_date, date_sub, add_months, col, when, coalesce, trim, broadcast, avg, max, min, lit, concat, window, round as colround, upper, abs as sparkabs
from pyspark.sql import functions as F
from pyspark import StorageLevel
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.window import Window

#Mostrar los duplicados
def print_res(sparkf_df):
  import pandas as pd
  pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)
  pd_spark_df = sparkf_df.toPandas()
 
  return pd_spark_df

In [0]:
# Configuraciones iniciales
from pyspark.sql import SparkSession
base_seguimiento = spark.sql("select distinct * from catalog_lhcl_prod_bcp.bcp_edv_fabseg.t39290_hm_app_hipahlo_f")

base_seguimiento = base_seguimiento.withColumn("CODMES", col("CODMES").cast("integer"))
base_seguimiento = base_seguimiento.withColumn("DEF5", when(col("DISTANCIA_DEF24") <= 5, lit(1)).otherwise(lit(0))) 

base_seguimiento.persist(StorageLevel.MEMORY_AND_DISK).count()

In [0]:
pd_cal='PD_2024Q4vf'#vigente
pd_rbm='PD_RBM'
pd_trc='PD_FINAL_BASE'
tipo_banda = 'Jeffrey'#Jeffrey Vasiseck
codmes_default = 202307
columna_monto='MTOAPROBADO_SOLES'
titulo='Calibración Applicant App Ahorro Local'
condiciones= "CODMES >= 201901 AND DEF_120_BLOQ_REF_0 = 0"

In [0]:
# Objeto de Seguimiento
Obj_Seg = rmm.MonitorScore(
  nombre_modelo         = 'Personas Modelo Applicant Ahorro Local',
  codigo_modelo         = 'MOD-BCP-20658',
  tier                  = 'II',
  detalle_seguimiento   = 'Seguimiento Applicant Ahorro Local',
  mes_seguimiento       = '202504',
  base_monitoreo        = base_seguimiento,
  pd1                   = pd_cal,
  pd2                   = pd_cal,
  monto_credito         = columna_monto,
  query_univ            = "DEF_120_BLOQ_REF_0=0 AND PD_2024Q4vf IS NOT NULL AND FLG_VC_VIVIENDA_RBM=1",
  bandas                = tipo_banda,
  codmes_default        = codmes_default,
  meses_ventana_target  = 24,
  meses_atraso_target   = 4
)

# 1. Calibración en pauta reciente

In [0]:
Obj_Seg_cal1, rc_1, rm_1, fc_1, fm_1 = Obj_Seg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "CODMES>=202101 AND PRS_ValorComVivienda<=239800",
  pd_seg             = pd_cal,
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 0,
  rd_aux             = 'RD21',
  rd_aux2            = 'RD18', # Corregido
  mora_temp          = 'MORA_30_3',

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = "met1_xc",
  suav_def_temp_xc   = 1,
  proy_mor_temp_xc   = 'det2_xc',
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14,

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = "np.where(met2_xm == 0, met2_xm + 0.008, met2_xm*1.25)",
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det3_xm',
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,

  # Detalles del gráfico de calibración
  titulo             = titulo,
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.2,
  eje_y_xm           = 0.2,
  dim_grafico        = (27,6.5),
  punt_mora          = 85, 
  etiquetas          =True,
  pos_etiquetas_xm   =10,
  pos_etiquetas_xc   =10,
  tamaño_etiqueta    =14,

  # Exportar factores para vista driver
  exportar_factores  = True
)

In [0]:
Obj_Seg_cal2, rc_2, rm_2, fc_2, fm_2 = Obj_Seg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "CODMES>=202101 AND PRS_ValorComVivienda>239800",
  pd_seg             = pd_cal,
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 0,
  rd_aux             = 'RD21',
  rd_aux2            = 'RD18', # Corregido
  mora_temp          = 'MORA_30_3',

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = "met1_xc",
  suav_def_temp_xc   = 1,
  proy_mor_temp_xc   = 'det2_xc',
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14,

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = "np.where(met2_xm == 0, met2_xm + 0.009, np.where(calxmontos['CODMES']==202309, met1_xm, met2_xm*1.25))",
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det3_xm',
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,

  # Detalles del gráfico de calibración
  titulo             = titulo,
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.2,
  eje_y_xm           = 0.2,
  dim_grafico        = (27,6.5),
  punt_mora          = 85, 
  etiquetas          =True,
  pos_etiquetas_xm   =10,
  pos_etiquetas_xc   =10,
  tamaño_etiqueta    =14,

  # Exportar factores para vista driver
  exportar_factores  = True,
)

In [0]:
Obj_Seg_cal, rc, rm, fc, fm = Obj_Seg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "CODMES>=202101",
  pd_seg             = pd_cal,
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 0,
  rd_aux             = 'RD21',
  rd_aux2            = 'RD18', # Corregido
  mora_temp          = 'MORA_30_3',

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = "",
  suav_def_temp_xc   = 1,
  proy_mor_temp_xc   = '',
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14,
  fact_to_proy_xc    = [fc_1, fc_2],

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = "",
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = '',
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,
  fact_to_proy_xm    = [fm_1, fm_2],

  # Detalles del gráfico de calibración
  titulo             = titulo,
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.125,
  eje_y_xm           = 0.125,
  dim_grafico        = (27,6.5),
  punt_mora          = 85, 
  etiquetas          =True,
  pos_etiquetas_xm   =10,
  pos_etiquetas_xc   =10,
  tamaño_etiqueta    =14,

  # Exportar factores para vista driver
  exportar_factores  = True
)

In [0]:
rm

In [0]:
rc

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("FLG_SEGMENTO_PRECIO", when(col("PRS_ValorComVivienda")>239800,1).otherwise(0))
driver = 'FLG_SEGMENTO_PRECIO'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202101,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.25),
  borde              = 0.08
)

# 2. Discriminación

In [0]:
var_pd = [pd_cal, pd_rbm]
caldri_veh_app = Obj_Seg.EvolDisVar(
  # Parametros de figura individual
  query_filtro = '',
  codmes_ini   = 202110,
  codmes_fin   = 202312,
  amplitud     = 6,
  disc_target  = [24, 21,18],
  umbral_sup   = 0.5,
  umbral_inf   = 0.4,
  moviles      = True,
  dim_grafico  = (22.5, 6),
  pos_leyenda  = (0.5, -0.35),
  etiquetas    = True,
  gini_abs     = False,
 
  # Parametros de figura final
  ncolumnas = 2,
  nfilas    = 1,
  variables = var_pd,
  dim_grafico_total = (20, 6),
  ncol_leyenda = 5,
  vspacio=-0.25,
  hspacio=4,
  tamaño_etiqueta = 14 
)

# 3. Seguimiento de variables del modelo



| **Variable**                     | **Descripción**                                                                                      | **Peso** | **Sentido** |
|----------------------------------|--------------------------------------------------------------------------------------------------------|----------|-------------|
| XB_FINAL_BASE                    | Score base                                                                                             | 50.2%    | +           |
| PD_DISEF_D2                      | Score de default de Disposición de Efectivo                                                           | 47.3%    | +           |
| EXP_PCT_EVOL_SHIP_U3M_RT_U6M_J   | Ratio entre media de saldo de balance entre los últimos 3m y últimos 6m (excl. hipotecario)           | 6.2%     | +           |

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("EXP_PCT_EVOL_SHIP_U3M_RT_U6M", when(col("EXP_PCT_EVOL_SHIP_U3M_RT_U6M")<0, 0).otherwise(col("EXP_PCT_EVOL_SHIP_U3M_RT_U6M")))
var_cal = ['XB_FINAL_BASE_REP', "EXP_PCT_EVOL_SHIP_U3M_RT_U6M", "PD_DISEF"]
var_cal_Q = Obj_Seg.CutVar(variables=var_cal, num_cortes=4, query_corte="CODMES BETWEEN 202103 AND 202208 AND DEF_120_BLOQ_REF_0 = 0")

In [0]:
driver = 'PD_DISEF_Q'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202101,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202504,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.25),
  borde              = 0.08,
  metrica_secundaria='Mediana'
)

In [0]:
var_cal
caldri = Obj_Seg.CalVar(
  # Parametros de figura individual
  query_uso          = "CODMES BETWEEN 202207 AND 202307",
  pd_seg             = pd_cal,
  rd_aux             = 'RD12',
  pd_comparacion     = True,
  factores_cuentas   = fc,
  factores_montos    = fm,
  mat_importancia    = 5, 
  cast_int           = True,
  dim_grafico        = (25, 6),
  pos_leyenda        = (0.5, -0.3),
  punt_mora          = 70,
 
  # Parametros de figura final
  tipo_vista         = 'M',
  ncolumnas          = 3,
  nfilas             = 1,
  variables          = var_cal_Q,
  dim_grafico_total  = (25, 5),
  ncol_leyenda       = 5,
  etiquetas_total    = True, 
  pos_etiqueta       = 0.05,
  vspacio            = -0.5,
  hspacio            = 0.5,
  filtro_var         ="not like '%issing%'",
  iv                 = True 
)

In [0]:
psi_dri = Obj_Seg.MixPSIVar(
  # Parametros de figura individual
    query_uso         ='CODMES >= 202011',
    cast_int          =True,
    codmes_inicio1    =202011,
    codmes_fin1       =202204,
    codmes_inicio2    =202401,
    codmes_fin2       =202504,
    dim_grafico       =(25, 6),
    pos_leyenda       =(0.5, -0.35),
 
    # Parametros de figura final
    tipo_vista        = 'M',
    ncolumnas         = 3,
    nfilas            = 1,
    variables         = var_cal_Q,
    dim_grafico_total = (25, 5),
    ncol_leyenda      =5,
    tamaño_leyenda    = 8.25,
    vspacio           =-0.45,
    hspacio           =0,
    borde             = 0.08,
    # metrica_secundaria='Media'
)

In [0]:
gini = Obj_Seg.EvolDisVar(
  # Parametros de figura individual
    query_filtro = '',
    codmes_ini   = 202110,
    codmes_fin   = 202312,
    amplitud     = 6,
    disc_target  = [24, 21,18],
    umbral_sup   = 0.15,
    umbral_inf   = 0.05,
    #umbral_opc   = 0.1,
    moviles      = True,
    dim_grafico  = (25, 5),
    pos_leyenda  = (0.5, -0.35),
 
    etiquetas    = True,
    tamaño_etiqueta    =12,

    # Parametros de figura final
    ncolumnas = 3,
    nfilas    = 1,
    variables = var_cal,
    dim_grafico_total = (25, 5),
    ncol_leyenda      = 6,
    vspacio           =-0.25,
    hspacio           =5
  )

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("XB_OPE_NOCLI_CAL", col("XB_OPE_NOCLI_CAL").cast("double"))
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("XB_OPE_CLI_CAL", col("XB_OPE_CLI_CAL").cast("double"))

var_pd = ["XB_OPE_NOCLI_CAL", "XB_OPE_CLI_CAL"]
caldri_veh_app = Obj_Seg.EvolDisVar(
  # Parametros de figura individual
  query_filtro = 'FLAG_CLIENTE = 0',
  codmes_ini   = 202206,
  codmes_fin   = 202312,
  amplitud     = 6,
  disc_target  = [24, 21,18],
  umbral_sup   = 0.4,
  umbral_inf   = 0.3,
  moviles      = True,
  dim_grafico  = (22.5, 6),
  pos_leyenda  = (0.5, -0.35),
  etiquetas    = True,
  gini_abs     = False,
 
  # Parametros de figura final
  ncolumnas = 2,
  nfilas    = 1,
  variables = var_pd,
  dim_grafico_total = (20, 6),
  ncol_leyenda = 5,
  vspacio=-0.25,
  hspacio=4,
  tamaño_etiqueta = 14 
)

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("XB_COMP_CLI_CAL", col("XB_COMP_CLI_CAL").cast("double"))
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("XB_COMP_CLI_CAL", col("XB_COMP_CLI_CAL").cast("double"))

var_pd = ["XB_COMP_CLI_CAL", "XB_OPE_CLI_CAL_REP"]
caldri_veh_app = Obj_Seg.EvolDisVar(
  # Parametros de figura individual
  query_filtro = 'FLAG_CLIENTE = 1',
  codmes_ini   = 202206,
  codmes_fin   = 202312,
  amplitud     = 6,
  disc_target  = [24, 21,18],
  umbral_sup   = 0.4,
  umbral_inf   = 0.3,
  moviles      = True,
  dim_grafico  = (22.5, 6),
  pos_leyenda  = (0.5, -0.35),
  etiquetas    = True,
  gini_abs     = False,
 
  # Parametros de figura final
  ncolumnas = 2,
  nfilas    = 1,
  variables = var_pd,
  dim_grafico_total = (20, 6),
  ncol_leyenda = 5,
  vspacio=-0.25,
  hspacio=4,
  tamaño_etiqueta = 14 
)

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("XB_COMP_NOCLI_CAL", col("XB_COMP_NOCLI_CAL").cast("double"))
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("XB_COMP_CLI_CAL", col("XB_COMP_CLI_CAL").cast("double"))

var_pd = ["XB_COMP_NOCLI_CAL", "XB_OPE_NOCLI_CAL_REP"]
caldri_veh_app = Obj_Seg.EvolDisVar(
  # Parametros de figura individual
  query_filtro = 'FLAG_CLIENTE = 0',
  codmes_ini   = 202206,
  codmes_fin   = 202312,
  amplitud     = 6,
  disc_target  = [24, 21,18],
  umbral_sup   = 0.4,
  umbral_inf   = 0.3,
  moviles      = True,
  dim_grafico  = (22.5, 6),
  pos_leyenda  = (0.5, -0.35),
  etiquetas    = True,
  gini_abs     = False,
 
  # Parametros de figura final
  ncolumnas = 2,
  nfilas    = 1,
  variables = var_pd,
  dim_grafico_total = (20, 6),
  ncol_leyenda = 5,
  vspacio=-0.25,
  hspacio=4,
  tamaño_etiqueta = 14 
)

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("XB_OPE_NOCLI_CAL", col("XB_OPE_NOCLI_CAL").cast("double"))
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("XB_OPE_CLI_CAL", col("XB_OPE_CLI_CAL").cast("double"))

var_pd = ["XB_FINAL_BASE_REP", "XB_FINAL_BASE"]
caldri_veh_app = Obj_Seg.EvolDisVar(
  # Parametros de figura individual
  query_filtro = '',
  codmes_ini   = 202206,
  codmes_fin   = 202312,
  amplitud     = 6,
  disc_target  = [24, 21,18],
  umbral_sup   = 0.4,
  umbral_inf   = 0.3,
  moviles      = True,
  dim_grafico  = (22.5, 6),
  pos_leyenda  = (0.5, -0.35),
  etiquetas    = True,
  gini_abs     = False,
 
  # Parametros de figura final
  ncolumnas = 2,
  nfilas    = 1,
  variables = var_pd,
  dim_grafico_total = (20, 6),
  ncol_leyenda = 5,
  vspacio=-0.25,
  hspacio=4,
  tamaño_etiqueta = 14 
)

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "LTV_CORREGIDO_VF", 
    when(col("CODMES") >= 202308, col("NFT_LTV")).otherwise(col("LTV_CORREGIDO_0"))
)

dri_neg = ["NFT_LTV", "PRS_TASA"]
dri_neg_Q = Obj_Seg.CutVar(variables=dri_neg, num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202412 AND DEF_120_BLOQ_REF_0 = 0")

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "RANGO_MTOVIVI_F",
    when(col("PRS_ValorComVivienda").isNull(), "99. Missing")
    .when(col("PRS_ValorComVivienda") <= 67400, "1. [0 - 67K]")
    .when(col("PRS_ValorComVivienda") <= 96200, "2. <67 - 96K]")
    .when(col("PRS_ValorComVivienda") <= 144000, "3. <96 - 144K]")
    .when(col("PRS_ValorComVivienda") <= 239800, "4. <144 - 239K]")
    .when(col("PRS_ValorComVivienda") <= 355100, "5. <239 - 355K]")
    .when(col("PRS_ValorComVivienda") <= 479300, "6. <355 - 479K]")
    .otherwise("7. <479K - ++]")
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("RNG_CEM6_MTO", 
    F.when(F.col("CEM6_MTO_T").isNull(), "99. Missing")
    .when(F.col("CEM6_MTO_T") <= 0.000430813, "01. [0 - 0.04%]")
    .when(F.col("CEM6_MTO_T") <= 0.0018687401, "02. <0.04- 0.18%]")
    .when(F.col("CEM6_MTO_T") <= 0.0041421573, "03. <0.18 - 0.4%]")
    .otherwise("04. <0.4 a mas]"))

In [0]:
dri_neg_Q = ["RANGO_MTOVIVI_F","NFT_LTV_Q","RNG_CEM6_MTO","EST_CIVIL_R","RANGO_MONTO", "RAN_PLAZO", "PRS_TASA_Q",  "MARCA_HML_T", "EDAD_R"]
caldri = Obj_Seg.CalVar(
  # Parametros de figura individual
  query_uso          = "CODMES >= 202212 AND CODMES <= 202304",
  pd_seg             = pd_cal,
  rd_aux             = 'RD12',
  pd_comparacion     = True,
  factores_cuentas   = fc,
  factores_montos    = fm,
  mat_importancia    = 5, 
  cast_int           = True,
  dim_grafico        = (25, 6),
  pos_leyenda        = (0.5, -0.3),
  punt_mora          = 70,
 
  # Parametros de figura final
  tipo_vista         = 'M',
  ncolumnas          = 3,
  nfilas             = 3,
  variables          = dri_neg_Q,
  dim_grafico_total  = (25, 15),
  ncol_leyenda       = 5,
  etiquetas_total    = True, 
  pos_etiqueta       = 0.05,
  vspacio            = -0.5,
  hspacio            = 0.5,
  filtro_var         ="not like '%issing%'",
  iv                 = True 
)

In [0]:
psi_dri = Obj_SegTotal.MixPSIVar(
  # Parametros de figura individual
    query_uso       ='CODMES >= 202201 AND CODMES <> 202407',
    cast_int        =True,
    codmes_inicio1  =202101,
    codmes_fin1     =202112,
    codmes_inicio2  =202301,
    codmes_fin2     =202312,
    dim_grafico     =(22.5, 6),
    pos_leyenda     =(0.5, -0.35),
 
    # Parametros de figura final
    tipo_vista      = 'M',
    ncolumnas       = 3,
    nfilas          = 3,
    variables       = dri_neg_Q,
    dim_grafico_total = (25, 15),
    ncol_leyenda    =8,
    tamaño_leyenda  = 8.25,
    vspacio         =-0.25,
    hspacio         =0,
    borde           = 0.08
)

In [0]:
dri_neg = ["NFT_LTV"]
dri_neg_Q = Obj_SegTotal.CutVar(variables=dri_neg, num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202412 AND DEF_120_BLOQ_REF_0 = 0")
Obj_SegTotal.base_monitoreo = Obj_SegTotal.base_monitoreo.withColumn("FLG_SEGMENTO_PRECIO", when(col("PRS_ValorComVivienda")>239800,1).otherwise(0))
driver = 'LTV_VF_R'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_SegTotal.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202101,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.25),
  borde              = 0.08
)

In [0]:
dri_neg = ["NFT_LTV"]
dri_neg_Q = Obj_SegTotal.CutVar(variables=dri_neg, num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202412 AND DEF_120_BLOQ_REF_0 = 0")
Obj_SegTotal.base_monitoreo = Obj_SegTotal.base_monitoreo.withColumn("FLG_SEGMENTO_PRECIO", when(col("PRS_ValorComVivienda")>239800,1).otherwise(0))
driver = 'NFT_LTV_Q'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_SegTotal.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202101,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.25),
  borde              = 0.08
)

In [0]:
driver = 'LTV_CORREGIDO_VF_Q'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_SegTotal.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202101,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.25),
  borde              = 0.08
)

In [0]:
pd_cal='MTOAPROBADO_SOLES'#vigente
pd_rbm='MTOAPROBADO_SOLES'
pd_trc='MTOAPROBADO_SOLES'
tipo_banda = 'Jeffrey'#Jeffrey Vasiseck
codmes_default = 202304
columna_monto='MTOAPROBADO_SOLES'
titulo='Calibración Applicant App Ahorro Local'
condiciones= "CODMES >= 201901 AND DEF_120_BLOQ_REF_0 = 0"

In [0]:
tabla = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_HM_ADM_HIP_AHLO_202504_F')
# Objeto de Seguimiento
Obj_SegTotal = rmm.MonitorScore_v01(
  nombre_modelo         = 'Personas Modelo Applicant Ahorro Local',
  codigo_modelo         = 'MOD-BCP-20658',
  tier                  = 'II',
  detalle_seguimiento   = 'Seguimiento Applicant Ahorro Local',
  mes_seguimiento       = '202504',
  base_monitoreo        = tabla,
  pd1                   = pd_cal,
  pd2                   = pd_trc,
  pd3                   = pd_rbm,
  monto_credito         = columna_monto,
  query_univ            = "DEF_120_BLOQ_REF_0=0 AND --PD_2024Q4vf IS NOT NULL AND FLG_VC_VIVIENDA_RBM=1",
  bandas                = tipo_banda,
  codmes_default        = codmes_default,
  meses_ventana_target  = 24,
  meses_atraso_target   = 4
)

In [0]:
Obj_SegTotal.base_monitoreo = Obj_SegTotal.base_monitoreo.withColumn(
    "LTV_CORREGIDO_VF", 
    when(col("CODMES") >= 202308, col("NFT_LTV")).otherwise(col("LTV_CORREGIDO_0"))
)

dri_neg = ["LTV_CORREGIDO_VF"]
dri_neg_Q = Obj_SegTotal.CutVar(variables=dri_neg, num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202412 AND DEF_120_BLOQ_REF_0 = 0")
Obj_SegTotal.base_monitoreo = Obj_SegTotal.base_monitoreo.withColumn("FLG_SEGMENTO_PRECIO", when(col("PRS_ValorComVivienda")>239800,1).otherwise(0))
driver = 'LTV_CORREGIDO_VF_Q'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_SegTotal.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202101,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.25),
  borde              = 0.08
)